In [1]:
import pandas as pd
import numpy as np
import jieba
import jieba.posseg as pseg
import wordcloud
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('weibo.xlsx')
df=df[~(df['时间'].isnull())]    #删掉空行
df = df.sort_values(by='时间')
df

,时间,排名,热搜内容,地域,主题,上榜时间,最后时间,Unnamed: 7,Unnamed: 8,Unnamed: 9
897,2020-01-09 12:00:00,7.0,武汉不明原因肺炎病原体为新型冠状病毒,国内-湖北-武汉,疫情-知识,2020/1/9\n17:36,2020-01-09 08:38:00,NaN,NaN,NaN
896,2020-01-11 12:00:00,48.0,专家解读武汉不明原因肺炎,国内,专家|疫情-知识,2020-01-11 14:50:00,2020/1/11\n8:18,NaN,NaN,NaN
895,2020-01-14 12:00:00,8.0,泰国发现1例新型冠状病毒病例,国外-泰国,疫情-新增,2020-01-14 20:30:00,2020/1/14\n11:42,NaN,NaN,NaN
894,2020-01-15 12:00:00,14.0,武汉肺炎不排除有限人传人可能,国内-湖北-武汉,疫情-知识,2020-01-15 13:40:00,2020/1/15\n7:52,NaN,NaN,NaN
893,2020-01-15 12:00:00,8.0,泰国新型冠状病毒病例为武汉游客,国外-泰国,疫情-新增,2020-01-15 15:00:00,2020/1/15\n8:04,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
24,2020-02-21 12:00:00,44.0,西藏连续22天无新增病例,国内-西藏,疫情-稳定,2020-02-21 12:18:00,2020/2/21\n11:48,NaN,NaN,NaN
26,2020-02-21 12:00:00,48.0,威斯特丹号全员检测阴性,国外-游轮,疫情-新增|资讯-威斯特丹号,2020-02-21 12:04:00,2020/2/21\n11:12,NaN,NaN,NaN
27,2020-02-21 12:00:00,49.0,河南学校3月1日后开学,国内-河南,复工-开学,2020-02-21 12:38:00,2020/2/21\n7:20,NaN,NaN,NaN
14,2020-02-21 12:00:00,25.0,浙江新增28例新型肺炎,国内-浙江,疫情-新增,2020-02-21 13:52:00,2020/2/21\n9:38,NaN,NaN,NaN


In [3]:
theme = df['主题']
content = df['热搜内容']
location = df['地域']
time = df['时间']

In [4]:
entity_list = ['新型冠状病毒/8/nz', '冠状病毒/8/nz','疫情地图/8/nz','武汉肺炎/8/nz','人传人','病毒/8/nz','疫情/8/nz',
               '病例','新冠/8/nz', 'SARS/8/nz','隔离', '口罩', '卫健委/5/nt','双黄连/8/nz','埃博拉/3/nz','瑞德西韦/5/nz',
           '金银潭/5/nz','百步亭/5/nz','火神山/5/nz','雷神山/5/nz','张家界/5/ns', '小汤山/5/nz','威斯特丹号/5/nz','钻石公主号/5/nz',
               '方舱医院/5/nz','同济医院/3/nz','协和医院/3/nz','紫外线', '蝙蝠/3/nz','酒精','消毒','确诊','连闯/2/v',
               '康复/3/n','封城/3/n','汤圆/3/n', '武软/3/nt', '红十字会/3/nt', '马拉松/3/n', '小姐姐/3/r',
               '医护人员/5/nr','医疗队/5/nr','医院院长/5/nr','张定宇/5/nr','护士/5/nr','医生/5/nr',
               '钟南山/5/nr', '李兰娟/5/nr','李文亮/5/nr', '应勇/5/nr','王贺胜/5/nr', '王忠林/5/nr']
for w in entity_list:
    w_list = w.split('/')
    freq, tag = 1, None
    if len(w_list) > 1:
        freq = w_list[1]    # [word freq]
    if len(w_list) > 2:
        tag = w_list[2] # [word freq tag]
    #jieba.add_word(w)
    jieba.add_word(w_list[0], freq=freq, tag=tag)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lxy\AppData\Local\Temp\jieba.cache
Loading model cost 1.694 seconds.
Prefix dict has been built successfully.


In [5]:
# 总体字典
epidemic_v = {}
epidemic_n = {}
gov_v = {}
gov_n = {}

# 每日列表
daily_epidemic_v = []
daily_epidemic_n = []
daily_gov_v = []
daily_gov_n = []

now_time = time.iloc[0]
for i in range(len(theme)):
    theme_i = theme.iloc[i][:2]
    content_i = content.iloc[i]
    location_i = location.iloc[i]
    time_i = str(time.iloc[i]).split(" ")[0]
    if time_i != now_time:
        # 如果是同一天则加入同一个每日列表
        # 如果不是同一天则将该行数组拼接为字符串作为字典值
        epidemic_v[time_i] = ' '.join(daily_epidemic_v)
        epidemic_n[time_i] = ' '.join(daily_epidemic_n)
        gov_v[time_i] = ' '.join(daily_gov_v)
        gov_n[time_i] = ' '.join(daily_gov_n)
        # 然后清空每日列表
        daily_epidemic_v = []
        daily_epidemic_n = []
        daily_gov_v = []
        daily_gov_n = []

    words = pseg.cut(content_i)
    for word, lexical in words:
        if len(word) > 1:
            if '疫情' in theme_i:
                if lexical.startswith('n'):
                    daily_epidemic_n.append(word)
                if lexical.startswith('v'):
                    daily_epidemic_v.append(word)
            if '政府' in theme_i:
                if lexical.startswith('n'):
                    daily_gov_n.append(word)
                if lexical.startswith('v'):
                    daily_gov_v.append(word)
    now_time = time_i

In [8]:
# 所有疫情主题词云
text_epidemic_v_all = ' '.join(list(epidemic_v.values()))
font = 'C:\\Windows\\Fonts\\YaHei Consolas Hybrid 1.12.ttf'
mask = plt.imread('china.jpg')
wc = wordcloud.WordCloud(background_color='white', font_path=font, mask=mask, max_words=20)
wc.generate(text_epidemic_v_all)
wc.to_file('疫情-动词-全部.jpg')